## Part (I/II): Affine registration

In [15]:
import os
import SimpleITK as sitk

def register_affine(fixed_image_path, moving_image_path):
    """
    Perform affine registration between two images using SimpleITK.
    
    Args:
    - fixed_image_path: Path to the fixed image (reference image).
    - moving_image_path: Path to the moving image (image to be registered).
    
    Returns:
    - final_transform: The resulting affine transformation (SimpleITK Transform object).
    """
    # Read the fixed and moving images
    fixed_image = sitk.ReadImage(fixed_image_path, sitk.sitkFloat32)
    moving_image = sitk.ReadImage(moving_image_path, sitk.sitkFloat32)

    # Initialize the registration method
    registration_method = sitk.ImageRegistrationMethod()

    # Set up the similarity metric
    registration_method.SetMetricAsMeanSquares()  # Use MeanSquares similarity metric

    # Set up the interpolator
    registration_method.SetInterpolator(sitk.sitkLinear)  # Linear interpolation for registration

    # Initialize the transformation (Affine)
    initial_transform = sitk.CenteredTransformInitializer(
        fixed_image,
        moving_image,
        sitk.AffineTransform(fixed_image.GetDimension()),  # Use affine transformation
        sitk.CenteredTransformInitializerFilter.GEOMETRY
    )

    # Set the initial transform
    registration_method.SetInitialTransform(initial_transform, inPlace=False)

    # Set up the optimizer
    registration_method.SetOptimizerAsRegularStepGradientDescent(
        learningRate=1.0,
        minStep=1e-6,
        numberOfIterations=200,
        gradientMagnitudeTolerance=1e-6
    )
    registration_method.SetOptimizerScalesFromPhysicalShift()

    # Execute the registration
    final_transform = registration_method.Execute(fixed_image, moving_image)

    # Output optimizer and metric information
    print("Optimizer's stopping condition: {0}".format(registration_method.GetOptimizerStopConditionDescription()))
    print("Final metric value: {0}".format(registration_method.GetMetricValue()))

    # Return the final transform (affine transformation)
    return final_transform


# Example usage of the registration function
data_dir = os.path.join(os.getcwd(), 'data/segthor_train/train/Patient_27')

fixed_image_path = os.path.join(data_dir, "isolated_heart_GT2.nii.gz")
moving_image_path = os.path.join(data_dir, "isolated_heart_GT.nii.gz")

# Perform the affine registration and get the transformation
affine_transform = register_affine(fixed_image_path, moving_image_path)

# Save the transformation to a file for future use
output_transform_path = os.path.join(data_dir, "affine_transform.tfm")
sitk.WriteTransform(affine_transform, output_transform_path)
print(f"Affine transform saved at: {output_transform_path}")


Optimizer's stopping condition: RegularStepGradientDescentOptimizerv4: Maximum number of iterations (200) exceeded.
Final metric value: 0.001245705170602879
Affine transform saved at: C:\Users\maart\Documents\ai4mi_project\data/segthor_train/train/Patient_27\affine_transform.tfm


## Part (II/II): Affine Transformation on All Patients

Running the following cell displays the retrieved affine matrix

In [17]:
import SimpleITK as sitk
import numpy as np

def display_affine_transform_matrix(transform_file):
    """
    Load and display the affine transformation as a matrix from a file.
    If the transformation is composite, extract the affine component.
    
    Args:
    - transform_file: Path to the file containing the affine transformation.
    
    Returns:
    - None
    """
    # Load the composite transformation from the file
    transform = sitk.ReadTransform(transform_file)
    
    # Check if the transform is a composite transform and extract affine component if present
    if isinstance(transform, sitk.CompositeTransform):
        for i in range(transform.GetNumberOfTransforms()):
            current_transform = transform.GetNthTransform(i)
            if isinstance(current_transform, sitk.AffineTransform):
                transform = current_transform
                break
    
    # Now check if we have an affine transform
    if isinstance(transform, sitk.AffineTransform):
        # Extract the affine matrix (rotation and scaling part)
        matrix = np.array(transform.GetMatrix()).reshape((3, 3))

        # Extract the translation vector
        translation = np.array(transform.GetTranslation())

        # Display the affine matrix and translation vector
        print("Affine transformation matrix (3x3):")
        print(matrix)
        print("\nTranslation vector:")
        print(translation)
    else:
        print("No affine transformation found in the file.")

# Example usage:
transform_file = "C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train/Patient_27/affine_transform.tfm"
display_affine_transform_matrix(transform_file)


Affine transformation matrix (3x3):
[[ 1.0107339   0.02432257  0.02479276]
 [ 0.11594829  1.00908553 -0.05903886]
 [-0.04915657 -0.01680958  0.98663108]]

Translation vector:
[-64.35096518 -43.02182351 -29.47999887]


Running this cell will create new files in each patients' directory with the transformation applied:

In [9]:
import os
import SimpleITK as sitk
import numpy as np

# Define the directory where patient folders are located
data_dir = "C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train"
# Path to the affine transform file
transform_file = "C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train/Patient_27/affine_transform.tfm"

# Function to load affine transformation from a file
def load_affine_transform(transform_file):
    """
    Loads an affine transformation from a file.

    Args:
    - transform_file: Path to the file containing the affine transformation.

    Returns:
    - transform: SimpleITK AffineTransform object.
    """
    transform = sitk.ReadTransform(transform_file)
    return transform

# Function to apply affine transformation to the heart (class 2)
def apply_affine_transform_to_heart(input_image_path, output_image_path, transform, heart_label=2):
    """
    Applies an affine transformation only to the heart (class 2) in a segmentation image.

    Args:
    - input_image_path: Path to the input segmentation image (GT.nii.gz).
    - output_image_path: Path to save the transformed image.
    - transform: Affine transformation object (SimpleITK Transform).
    - heart_label: Label corresponding to the heart class (default is 2).

    Returns:
    - None
    """
    # Read the input image (GT.nii.gz)
    input_image = sitk.ReadImage(input_image_path)
    
    # Convert the input image to a numpy array
    input_array = sitk.GetArrayFromImage(input_image)
    
    # Isolate the heart (class 2) by creating a binary mask
    heart_mask = (input_array == heart_label).astype(np.uint8)
    
    # Convert the heart mask to a SimpleITK image
    heart_image = sitk.GetImageFromArray(heart_mask)
    heart_image.CopyInformation(input_image)
    
    # Apply the affine transformation only to the heart
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(heart_image)
    resampler.SetTransform(transform)
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)  # Nearest neighbor to preserve label values
    
    transformed_heart_image = resampler.Execute(heart_image)
    
    # Convert the transformed heart back to a numpy array
    transformed_heart_array = sitk.GetArrayFromImage(transformed_heart_image)
    
    # Reintegrate the transformed heart back into the original segmentation
    output_array = input_array.copy()
    output_array[transformed_heart_array == 1] = heart_label
    
    # Convert the output array back to a SimpleITK image
    output_image = sitk.GetImageFromArray(output_array)
    output_image.CopyInformation(input_image)
    
    # Write the transformed image to the output file
    sitk.WriteImage(output_image, output_image_path)
    print(f"Transformed heart saved at: {output_image_path}")

# Loop through all patient directories
def process_patients(data_dir, transform_file):
    """
    Applies affine transformation to the heart for all patient segmentation images.

    Args:
    - data_dir: Path to the root directory containing patient folders.
    - transform_file: Path to the file containing the affine transformation.

    Returns:
    - None
    """
    # Load the affine transformation from file
    transform = load_affine_transform(transform_file)
    
    for patient_folder in os.listdir(data_dir):
        patient_path = os.path.join(data_dir, patient_folder)
        
        # Check if the directory contains the GT.nii.gz file
        gt_path = os.path.join(patient_path, "GT.nii.gz")
        
        if os.path.isfile(gt_path):
            # Define the output path for the transformed image
            output_path = os.path.join(patient_path, "GT_transformed.nii.gz")
            
            # Apply the transformation to the heart and save the result
            apply_affine_transform_to_heart(gt_path, output_path, transform)
        else:
            print(f"GT.nii.gz not found for {patient_folder}")


# Apply affine transformation to all patients
process_patients(data_dir, transform_file)


Optimizer's stopping condition: RegularStepGradientDescentOptimizerv4: Maximum number of iterations (200) exceeded.
Final metric value: 0.001245705170602879
Transformed image saved at: C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train/Patient_27/transformed_image.nii.gz
